In [1]:
import numpy as np
import pandas as pd

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.2.2 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [57]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [63]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [186]:
import sklearn
from sklearn import model_selection

In [74]:
import re

In [184]:
import time

In [2]:
#Import Data
train_data=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')
#attributes=pd.read_csv('attributes.csv')
description=pd.read_csv('product_descriptions.csv')

In [8]:
train_data=train_data.drop('id',axis=1)

In [11]:
train_vec=pd.merge(train_data,description,left_on='product_uid',right_on='product_uid')

In [12]:
train_vec.head()

,product_uid,product_title,search_term,relevance,product_description
0,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ..."
1,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ..."
2,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...
3,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...
4,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Update your bathroom with the Delta Vero Singl...


In [131]:
def clean_text(sentence):
    sentence = unicode(sentence,'utf-8', errors='replace')
    cleaned=' '.join([stemmer.stem(w) for w in sentence.split()])
    cleaned=' '.join([w for w in cleaned.split() if w not in stop])
    cleaned=re.sub('\W',' ',cleaned)
    return cleaned

In [153]:
def my_tf_df(sentence, docu):
    wf=[]
    for word in sentence.split():
        wf.append((np.array(docu.split())==[word]).sum()/1.0/len(docu.split()))
    return np.array(wf).mean()

In [155]:
train_vec.search_term=train_vec.search_term.apply(clean_text)

In [156]:
train_vec.product_description=train_vec.product_description.apply(clean_text)

In [157]:
train_vec.product_title=train_vec.product_title.apply(clean_text)

In [159]:
#spell correcting

from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter('\n'.join(s for s in train_vec.product_description.as_matrix().flatten()).split())

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [161]:
train_vec.search_term=train_vec.search_term.apply(lambda x: ' '.join([correction(w) for w in x.split()]))

In [163]:
train_vec.to_csv('myplayingwithdataset.csv')

In [165]:
train_vec.shape

(74067, 5)

In [178]:
to_train=pd.DataFrame(np.zeros([len(train_vec),2]),columns=['Title','Desc'])

In [179]:
for i in range(len(train_vec)):
    to_train['Title'][i]=my_tf_df(train_vec['search_term'][i],train_vec['product_title'][i])
    to_train['Desc'][i]=my_tf_df(train_vec['search_term'][i],train_vec['product_description'][i])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice.
  """
/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [180]:
to_train

,Title,Desc
0,0.083333,0.015306
1,0.000000,0.000000
2,0.000000,0.007937
3,0.027778,0.004630
4,0.083333,0.009259
5,0.041667,0.005797
6,0.041667,0.005797
7,0.083333,0.011594
8,0.150000,0.030612
9,0.095238,0.022599


In [181]:
import xgboost as xgb

In [182]:
param={}
param['eta']=0.01
param['max_depth']=6
param['silent']=1
param['eval_metric']='rmse'
param['min_child_weight']=3
param['subsample']=0.7
param['colsample_bytree']=0.7
num_rounds=50000

In [187]:
start_ = time.time()

#x_train = np.array(train_vec.iloc[0:50000])
#y_train = np.array(Relevance.iloc[0:50000])

#x_validation = np.array(train_vec.iloc[50000:])
#y_validation = np.array(Relevance.iloc[50000:])
x_train, x_validation, y_train, y_validation=model_selection.train_test_split(to_train,train_vec['relevance'],test_size=0.3)

xgtrain = xgb.DMatrix(x_train, label= y_train)
xgvalidation=xgb.DMatrix(x_validation,label=y_validation)

In [188]:
clf = xgb.train(param, xgtrain, num_rounds,evals=[ (xgtrain,'train'),(xgvalidation,'eval')],
                early_stopping_rounds=100, verbose_eval =100)

[0]	train-rmse:1.93772	eval-rmse:1.93761
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 100 rounds.
[100]	train-rmse:0.852339	eval-rmse:0.851046
[200]	train-rmse:0.562295	eval-rmse:0.561576
[300]	train-rmse:0.509383	eval-rmse:0.509783
[400]	train-rmse:0.501108	eval-rmse:0.502317
[500]	train-rmse:0.499491	eval-rmse:0.501294
[600]	train-rmse:0.498894	eval-rmse:0.501183
Stopping. Best iteration:
[586]	train-rmse:0.498969	eval-rmse:0.501175

